In [24]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters import * 

In [25]:
data = io.load('data/gauss4')
# data = io.load('data/simulations')

In [26]:
import warnings

warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [27]:
from ipywidgets import Output
debug = Output(title='debug')
debug

Output()

### Helper functions

#### view

In [28]:
def update_view(view, f):
    def _inner():
        view.show = filter_tree(view.tree, f)
    return _inner 

In [29]:
def view(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=update_view(v, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

#### reduce

In [30]:
def reduce_view(widget, tree, f):
    @debug.capture()
    def _inner():
        widget.ref = reduce_tree(tree, filter=f)
    return _inner

In [31]:
def reduce(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data.tree, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=reduce_view(v, data.tree, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

### Initial tree

In [32]:
tw = TreeWidget(data.tree)

In [33]:
v1, f1, p1 = view(tw, title='Initial Tree')

In [34]:
v2, f2, p2 = view(tw, attr='fitness', title='fitness')

### Reduce tree

In [35]:
v2, r2, p2 = reduce(tw, attr='span', title='reduce')

In [36]:
v2.attr = 'parent_fitness'

### fitness

#### test

In [37]:
tw2 = TreeWidget(data.tree)

In [38]:
f = AttrFilter(attr='span', func=lambda x, v: v <= x)

In [39]:
def test():
    tw2.ref = reduce_tree(tw.tree, filter=f)

In [40]:
m = Monitor(f, func=test)

In [41]:
test_view = TreeView(tw2, attr='span') 

In [42]:
p = SidePanel(title='Reduce initial tree')
with p:
    display(test_view, f)

In [43]:
fitness = view(tw2, attr='fitness', title='fitness')

In [44]:
parent_fitness = view(tw2, attr='parent_fitness', title='parent_fitness')

### Details

In [56]:
dw = DataWidget(data=data)

In [57]:
details = DetailsView(data=dw)

In [58]:
p2 = SidePanel(title='details')
with p2:
    display(details)

In [59]:
l1 = widgets.link((fitness[0], 'details'), (details, 'show'))

In [60]:
l2 = widgets.link((parent_fitness[0], 'details'), (details, 'show'))